# Composability

This notebook shows off some basic functionality around LangChain Expression Language, which makes it really easy to compose arbitrary chains.

For a much deeper dive, see:

- [Full LangChain Expression Language Documentation](https://python.langchain.com/docs/expression_language/)

## Basic Composability

In [30]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import os
from dotenv import load_dotenv
from pathlib import Path
import openai

dotenv_path = Path('./.env')
load_dotenv(dotenv_path=dotenv_path)  # add your GOOGLE API key here

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
openai.api_key = os.getenv("OPEN_AI_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPEN_AI_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [20]:
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
# model = ChatOpenAI(api_key=os.getenv("OPEN_AI_KEY"))
model = ChatOpenAI()
googlellm  = ChatGoogleGenerativeAI(model="gemini-pro")
output_parser = StrOutputParser()

In [21]:
chain = prompt | model | output_parser
chain2 = prompt | googlellm | output_parser

In [14]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

In [22]:
chain2.invoke({"topic": "bears"})

"Why did the bear get kicked out of the restaurant?\n\nBecause he wasn't wearing pants!"

### Batching

In [18]:
chain.batch([{"topic": "bear"}, {"topic": "clowns"}])

["Why don't bears wear shoes?\n\nBecause they have bear feet!",
 'Why don\'t clowns like talking on the phone?\n\nBecause it always involves a lot of "ring-a-ling"!']

In [26]:
chain2.batch([{"topic": "bear"}, {"topic": "gold"}])

["Why did the bear get kicked out of the restaurant?\n\nBecause he ate the waiter's tips.",
 "Why don't pirates eat gold for breakfast?\n\nBecause it’s too “coin”cidental."]

### Streaming

In [27]:
for s in chain.stream({"topic": "bears"}):
    print(s)


Why
 don
't
 bears
 wear
 shoes
?


Because
 they
 have
 bear
 feet
!



In [28]:
for s in chain2.stream({"topic": "bears"}):
    print(s)

Why did the bear go to the doctor?
Because he wasn't feeling
 berry well.


## RunnablePassthrough

In [31]:
from langchain_community.retrievers.tavily_search_api import TavilySearchAPIRetriever

retriever= TavilySearchAPIRetriever()

prompt = ChatPromptTemplate.from_template("""Answer the question based only on the context provided:

Context: {context}

Question: {question}""")

In [32]:
chain = prompt | model | output_parser
chain2 = prompt | googlellm | output_parser

In [33]:
question = "what is langsmith"
context = "langsmith is a testing and observability platform built by the langchain team"
chain.invoke({"question": question, "context": context})

'Langsmith is a testing and observability platform developed by the Langchain team.'

In [35]:
chain2.invoke({"question": question, "context": context})

'A testing and observability platform built by the langchain team'

In [36]:
from langchain_core.runnables import RunnablePassthrough

retrieval_chain = RunnablePassthrough.assign(
    context=(lambda x: x["question"]) | retriever
) | chain

retrieval_chain2 = RunnablePassthrough.assign(
    context=(lambda x: x["question"]) | retriever
) | chain2

In [38]:
retrieval_chain.invoke({"question": "what is langsmith"})

'LangSmith is a platform that helps trace and evaluate language model applications and intelligent agents, assisting in moving from prototype to production. It provides features such as debugging, testing, and monitoring chains and agents built on any LLM framework, and integrates seamlessly with LangChain.'

In [39]:
retrieval_chain2.invoke({"question": "what is langsmith"})

'LangSmith is a platform that helps trace and evaluate language model applications and intelligent agents to move from prototype to production.'

## RunnableParallel

In [40]:
from langchain_core.runnables import RunnableParallel

In [41]:
prompt = ChatPromptTemplate.from_template("""{question}""")
simple_chain = prompt | model | output_parser
simple_chain2 = prompt | googlellm | output_parser

In [43]:
parallel_chain = RunnableParallel({
    "retrieved_answer": retrieval_chain,
    "simple_answer": simple_chain
})

parallel_chain2 = RunnableParallel({
    "retrieved_answer": retrieval_chain2,
    "simple_answer": simple_chain2
})

In [45]:
parallel_chain.invoke({"question": "what is langsmith"})

{'retrieved_answer': 'LangSmith is a platform that helps trace and evaluate language model applications and intelligent agents, allowing developers to move from prototype to production. It provides tools for debugging, testing, evaluating, and monitoring chains and agents built on any language model (LLM) framework. It seamlessly integrates with LangChain, an open-source framework for building with LLMs.',
 'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "Langsmith". It might be a misspelling or a lesser-known term. Can you please provide more context or clarify your question?'}

In [46]:
parallel_chain2.invoke({"question": "what is langsmith"})

{'retrieved_answer': 'LangSmith is a unified platform for debugging, testing, evaluating, and monitoring chains and intelligent agents built on any LLM framework. It seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.',
 'simple_answer': "Langsmith is a language learning platform that aims to make language learning more fun and engaging. It offers a variety of courses in different languages, as well as games, quizzes, and other activities to help learners stay motivated and engaged.\n\nLangsmith uses a variety of teaching methods to cater to different learning styles. These methods include:\n\n* **Interactive lessons:** Langsmith's lessons are interactive and engaging, with a variety of activities to keep learners entertained and motivated.\n* **Games and quizzes:** Langsmith offers a variety of games and quizzes to help learners practice their language skills in a fun and engaging way.\n* **Real-world content:** Langsmith's lessons and activit

In [47]:
for s in parallel_chain.stream({"question": "what is langsmith"}):
    print(s)

{'simple_answer': ''}
{'simple_answer': 'There'}
{'simple_answer': ' is'}
{'simple_answer': ' no'}
{'simple_answer': ' widely'}
{'simple_answer': ' recognized'}
{'simple_answer': ' term'}
{'simple_answer': ' or'}
{'simple_answer': ' definition'}
{'simple_answer': ' for'}
{'simple_answer': ' "'}
{'simple_answer': 'lang'}
{'simple_answer': 'smith'}
{'simple_answer': '."'}
{'simple_answer': ' It'}
{'simple_answer': ' could'}
{'simple_answer': ' potentially'}
{'simple_answer': ' be'}
{'simple_answer': ' a'}
{'simple_answer': ' proper'}
{'simple_answer': ' noun'}
{'simple_answer': ' referring'}
{'simple_answer': ' to'}
{'simple_answer': ' a'}
{'simple_answer': ' specific'}
{'simple_answer': ' person'}
{'simple_answer': ' or'}
{'simple_answer': ' entity'}
{'simple_answer': '.'}
{'simple_answer': ' Without'}
{'simple_answer': ' further'}
{'simple_answer': ' context'}
{'simple_answer': ' or'}
{'simple_answer': ' information'}
{'simple_answer': ','}
{'simple_answer': ' it'}
{'simple_answer': ' 

Connection error caused failure to post https://api.smith.langchain.com/runs  in LangSmith API. Please confirm your LANGCHAIN_ENDPOINT. ConnectTimeout(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x1486eae90>, 'Connection to api.smith.langchain.com timed out. (connect timeout=7.0)'))"))
Connection error caused failure to patch https://api.smith.langchain.com/runs/02c8b2ac-23cd-47c8-8473-67faa7413c58  in LangSmith API. Please confirm your LANGCHAIN_ENDPOINT. ConnectionError(MaxRetryError('HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /runs/02c8b2ac-23cd-47c8-8473-67faa7413c58 (Caused by ReadTimeoutError("HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Read timed out. (read timeout=7.0)"))'))


ReadTimeout: HTTPSConnectionPool(host='api.tavily.com', port=443): Read timed out. (read timeout=100)

In [17]:
result = {}
for s in parallel_chain.stream({"question": "what is langsmith"}):
    for k,v in s.items():
        if k not in result:
            result[k] = ""
        result[k] += v
    print(result)

{'simple_answer': ''}
{'simple_answer': 'I'}
{'simple_answer': "I'm"}
{'simple_answer': "I'm sorry"}
{'simple_answer': "I'm sorry,"}
{'simple_answer': "I'm sorry, but"}
{'simple_answer': "I'm sorry, but I"}
{'simple_answer': "I'm sorry, but I couldn"}
{'simple_answer': "I'm sorry, but I couldn't"}
{'simple_answer': "I'm sorry, but I couldn't find"}
{'simple_answer': "I'm sorry, but I couldn't find any"}
{'simple_answer': "I'm sorry, but I couldn't find any information"}
{'simple_answer': "I'm sorry, but I couldn't find any information on"}
{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "'}
{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "Lang'}
{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "Langsmith'}
{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "Langsmith."'}
{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "Langsmith." It'}
{'simple_answer': 'I\'m sorry, but I coul

{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "langsmith." It seems to be a term that is not widely recognized or used. Could you please provide more context or clarify your question?', 'retrieved_answer': 'LangSmith is a platform that offers features for debugging, testing, evaluating, and monitoring Language Learning Models (LLMs) and AI applications. It helps users trace and evaluate their language model applications and intelligent agents, providing visibility'}
{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "langsmith." It seems to be a term that is not widely recognized or used. Could you please provide more context or clarify your question?', 'retrieved_answer': 'LangSmith is a platform that offers features for debugging, testing, evaluating, and monitoring Language Learning Models (LLMs) and AI applications. It helps users trace and evaluate their language model applications and intelligent agents, providing visibility into'}
{'s